<a href="https://colab.research.google.com/github/COMS-BC3159-F24/colabs/blob/main/VectorSumCUDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Vector Sum in CUDA
First lets install some pre-requistes and make sure the runtime is correct!

In [ ]:
# make sure CUDA is installed
!nvcc --version

# make sure you have a GPU runtime (if this fails go to runtime -> change runtime type)
!nvidia-smi

# CUDA in Jupyter helpers
!pip install nvcc4jupyter
%load_ext nvcc4jupyter
# to learn about how to do more fancy things with CUDA using this API see:
# https://nvcc4jupyter.readthedocs.io/en/latest/index.html

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
Wed Sep  4 15:55:32 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8               9W /  7

Then our vector sum function!

In [ ]:
# some magic to make things work in Jupyter
%%cuda_group_save -n sum.h -g default

// vector sum wrapped in a function call
__global__
void vectorSum_kernel(float *d_data, const int c_dataLength){
  int levelSize = c_dataLength;
  while(levelSize > 1){
    for(int i= threadIdx.x; i<levelSize/2; i+= blockDim.x){
        d_data[i] += d_data[i+levelSize/2];
    }
    if (levelSize % 2){d_data[0] += d_data[levelSize-1];}
    levelSize /= 2;
    __syncthreads();
  }
}

Then wrapper code to launch and run it!

In [ ]:
%%cuda_group_save -n run.h -g default
#include "sum.h"

float vectorSum(float *h_data, const int c_dataLength){
    // move data to GPU
    float *d_data; cudaMalloc(&d_data,c_dataLength*sizeof(float));
    cudaMemcpy(d_data,h_data,c_dataLength*sizeof(float),cudaMemcpyHostToDevice);
    // run the kernel
    vectorSum_kernel<<<1,64,0,0>>>(d_data,c_dataLength);
    // move data back
    cudaDeviceSynchronize();
    float sum; cudaMemcpy(&sum,d_data,sizeof(float),cudaMemcpyDeviceToHost);
    return sum;
}

float serialSum(float *h_data, const int c_dataLength){
    float sum = 0.0;
    for(unsigned i = 0; i < c_dataLength; i++){
      sum += h_data[i];
    }
    return sum;
}

void runTest(float *h_data, const int c_dataLength){
    printf("--------------------------------------------------\n");
    // print the CPU answer
    float sumCPU = serialSum(h_data, c_dataLength);
    printf("CPU Sum = [%f] for Input of Length [%d]\n", sumCPU, c_dataLength);

    // run the GPU code
    float sumGPU = vectorSum(h_data,c_dataLength);
    printf("GPU Sum = [%f] for Input of Length [%d]\n", sumGPU, c_dataLength);
}

Now lets run some tests!

In [ ]:
%%cuda_group_save -n run.cu -g default

#include <cuda_runtime.h>
#include <stdlib.h>
#include <stdio.h>
#include <time.h>
#include "run.h"

const int max_size = 16;

int main() {
  // Seed the random number generator
  srand(time(NULL));

  // generate random arrays and test them
  for (unsigned i = 0; i < 10; i++){
      // Allocate memory for the array
      unsigned N = rand() % max_size + 1;
      float *data = (float *)malloc(N * sizeof(float));

      // Fill the array with random floating-point numbers
      for (int i = 0; i < N; i++) {
          data[i] = (float)rand() / RAND_MAX;  // Generates a random float between 0 and 1
      }

      runTest(data,N);
  }
}

In [ ]:
%cuda_group_run -g default

--------------------------------------------------
CPU Sum = [0.573053] for Input of Length [2]
GPU Sum = [0.573053] for Input of Length [2]
--------------------------------------------------
CPU Sum = [3.161889] for Input of Length [7]
GPU Sum = [2.792033] for Input of Length [7]
--------------------------------------------------
CPU Sum = [7.852239] for Input of Length [12]
GPU Sum = [8.123331] for Input of Length [12]
--------------------------------------------------
CPU Sum = [5.275385] for Input of Length [11]
GPU Sum = [6.996038] for Input of Length [11]
--------------------------------------------------
CPU Sum = [0.921286] for Input of Length [2]
GPU Sum = [0.921286] for Input of Length [2]
--------------------------------------------------
CPU Sum = [3.339063] for Input of Length [9]
GPU Sum = [3.310093] for Input of Length [9]
--------------------------------------------------
CPU Sum = [2.929603] for Input of Length [7]
GPU Sum = [3.417165] for Input of Length [7]
---------

Did the results surprise you? Do you remember from class what went wrong?

Let's implement the fix from the lecture slides!

In [ ]:
# overwrite the old sum kernel!
%%cuda_group_save -n sum.h -g default

// vector sum wrapped in a function call
__global__
void vectorSum_kernel(float *d_data, const int c_dataLength){
  int levelSize = c_dataLength;
  while(levelSize > 1){
    for(int i= threadIdx.x; i<levelSize/2; i+= blockDim.x){
        d_data[i] += d_data[i+levelSize/2];
    }
    if (levelSize % 2 && threadIdx.x == 0){d_data[0] += d_data[levelSize-1];}
    levelSize /= 2;
    __syncthreads();
  }
}

In [ ]:
%cuda_group_run -g default

--------------------------------------------------
CPU Sum = [6.622480] for Input of Length [11]
GPU Sum = [6.622480] for Input of Length [11]
--------------------------------------------------
CPU Sum = [1.730114] for Input of Length [3]
GPU Sum = [1.730114] for Input of Length [3]
--------------------------------------------------
CPU Sum = [8.237677] for Input of Length [14]
GPU Sum = [8.237677] for Input of Length [14]
--------------------------------------------------
CPU Sum = [3.333950] for Input of Length [6]
GPU Sum = [3.333950] for Input of Length [6]
--------------------------------------------------
CPU Sum = [2.388439] for Input of Length [4]
GPU Sum = [2.388439] for Input of Length [4]
--------------------------------------------------
CPU Sum = [5.752578] for Input of Length [14]
GPU Sum = [5.752579] for Input of Length [14]
--------------------------------------------------
CPU Sum = [1.716892] for Input of Length [5]
GPU Sum = [1.716892] for Input of Length [5]
-------

## Vector Sum 2.0
Now lets explore the alternative design from lecture! We'll need a few new functions to account for the additional input into the function, but can leave some the same.

In [ ]:
%%cuda_group_save -n sum2.h -g default2

// vector sum wrapped in a function call
__global__
void vectorSum_kernel2(float *d_data, float *d_sum, const int c_dataLength){
  for(int i = threadIdx.x; i < c_dataLength; i += blockDim.x){
        *d_sum += d_data[i];
  }
}

In [ ]:
%%cuda_group_save -n run2.h -g default2
#include "sum2.h"

float vectorSum2(float *h_data, const int c_dataLength){
    // move data to GPU
    float *d_data; cudaMalloc(&d_data,c_dataLength*sizeof(float));
    float *d_sum; cudaMalloc(&d_sum,sizeof(float));
    cudaMemcpy(d_data,h_data,c_dataLength*sizeof(float),cudaMemcpyHostToDevice);
    // run the kernel
    vectorSum_kernel2<<<1,64,0,0>>>(d_data,d_sum,c_dataLength);
    // move data back
    cudaDeviceSynchronize();
    float sum; cudaMemcpy(&sum,d_sum,sizeof(float),cudaMemcpyDeviceToHost);
    return sum;
}

float serialSum(float *h_data, const int c_dataLength){
    float sum = 0.0;
    for(unsigned i = 0; i < c_dataLength; i++){
      sum += h_data[i];
    }
    return sum;
}

void runTest2(float *h_data, const int c_dataLength){
    printf("--------------------------------------------------\n");
    // print the CPU answer
    float sumCPU = serialSum(h_data, c_dataLength);
    printf("CPU Sum = [%f] for Input of Length [%d]\n", sumCPU, c_dataLength);

    // run the GPU code
    float sumGPU = vectorSum2(h_data,c_dataLength);
    printf("GPU Sum = [%f] for Input of Length [%d]\n", sumGPU, c_dataLength);
}

In [ ]:
%%cuda_group_save -n run2.cu -g default2

#include <cuda_runtime.h>
#include <stdlib.h>
#include <stdio.h>
#include <time.h>
#include "run2.h"

const int max_size = 16;

int main() {
  // Seed the random number generator
  srand(time(NULL));

  // generate random arrays and test them
  for (unsigned i = 0; i < 10; i++){
      // Allocate memory for the array
      unsigned N = rand() % max_size + 1;
      float *data = (float *)malloc(N * sizeof(float));

      // Fill the array with random floating-point numbers
      for (int i = 0; i < N; i++) {
          data[i] = (float)rand() / RAND_MAX;  // Generates a random float between 0 and 1
      }

      runTest2(data,N);
  }
}

In [ ]:
%cuda_group_run -g default2

--------------------------------------------------
CPU Sum = [1.990526] for Input of Length [3]
GPU Sum = [0.842678] for Input of Length [3]
--------------------------------------------------
CPU Sum = [4.049982] for Input of Length [8]
GPU Sum = [0.110907] for Input of Length [8]
--------------------------------------------------
CPU Sum = [3.773655] for Input of Length [10]
GPU Sum = [0.190786] for Input of Length [10]
--------------------------------------------------
CPU Sum = [6.000062] for Input of Length [12]
GPU Sum = [0.938075] for Input of Length [12]
--------------------------------------------------
CPU Sum = [3.633538] for Input of Length [8]
GPU Sum = [0.845946] for Input of Length [8]
--------------------------------------------------
CPU Sum = [3.371860] for Input of Length [5]
GPU Sum = [0.138688] for Input of Length [5]
--------------------------------------------------
CPU Sum = [5.550526] for Input of Length [12]
GPU Sum = [0.479892] for Input of Length [12]
-------

Did the results surprise you? Do you remember from class what went wrong?

Let's implement the fix from the lecture slides!

In [ ]:
# overwrite the old sum kernel!
%%cuda_group_save -n sum2.h -g default2

// vector sum wrapped in a function call
__global__
void vectorSum_kernel2(float *d_data, float *d_sum, const int c_dataLength){
  for(int i = threadIdx.x; i < c_dataLength; i += blockDim.x){
      atomicAdd(d_sum,d_data[i]);
  }
}

In [ ]:
%cuda_group_run -g default2

--------------------------------------------------
CPU Sum = [6.821296] for Input of Length [16]
GPU Sum = [6.821296] for Input of Length [16]
--------------------------------------------------
CPU Sum = [5.863772] for Input of Length [11]
GPU Sum = [5.863772] for Input of Length [11]
--------------------------------------------------
CPU Sum = [1.442321] for Input of Length [4]
GPU Sum = [1.442321] for Input of Length [4]
--------------------------------------------------
CPU Sum = [2.199970] for Input of Length [5]
GPU Sum = [2.199970] for Input of Length [5]
--------------------------------------------------
CPU Sum = [0.872175] for Input of Length [2]
GPU Sum = [0.872175] for Input of Length [2]
--------------------------------------------------
CPU Sum = [2.078471] for Input of Length [7]
GPU Sum = [2.078471] for Input of Length [7]
--------------------------------------------------
CPU Sum = [0.960726] for Input of Length [2]
GPU Sum = [0.960726] for Input of Length [2]
---------